### <center>Signal Processing</center>

**Note**: _This file is only for testing purposes. It is not user-friendly and requires user to adjust it for themselves._


In [1]:
%pip install -r "D:\\natHacks\\NeuroKnights\\requirements.txt"

Note: you may need to restart the kernel to use updated packages.


#### Imports


In [ ]:
import time
import serial.tools.list_ports

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

#### Listing available ports


In [2]:
ports = serial.tools.list_ports.comports()
print([port.device for port in ports])

['COM3', 'COM6']


In [3]:
BoardShim.enable_dev_board_logger()

params = BrainFlowInputParams()

params.serial_port = "COM6"

board = BoardShim(BoardIds.NEUROPAWN_KNIGHT_BOARD, params)

In [4]:
print(board.get_sampling_rate(BoardIds.NEUROPAWN_KNIGHT_BOARD))
# print(board.get_eeg_channels(-1))

125


In [5]:
board.prepare_session()
print(board.is_prepared())

True


In [6]:
board.start_stream(450000)

In [19]:
print(
    f"Indecies for EEG channels {BoardShim.get_eeg_channels(BoardIds.NEUROPAWN_KNIGHT_BOARD)}"
)
print(
    f"Indecies for EXG channels {BoardShim.get_exg_channels(BoardIds.NEUROPAWN_KNIGHT_BOARD)}"
)

Indecies for EEG channels [1, 2, 3, 4, 5, 6, 7, 8]
Indecies for EXG channels [1, 2, 3, 4, 5, 6, 7, 8]


In [7]:
channels = list(range(1, 5))
gain_value = 12

In [30]:
for channel in channels:
    board.config_board(f"chon_{channel}_{gain_value}")
    print(f"Enabled channel {channel} with gain {gain_value}.")
    time.sleep(0.5)

Enabled channel 1 with gain 12.
Enabled channel 2 with gain 12.
Enabled channel 3 with gain 12.
Enabled channel 4 with gain 12.


In [34]:
assert board
data = board.get_current_board_data(1000)[1:5]
data

array([[ -38.14697647,  184.89363098,  127.75263977, ...,  457.12796021,
        -283.08236694,  566.7210083 ],
       [ -69.9758606 ,  196.61587524,   92.54615784, ...,  424.62353516,
        -271.08197021,  543.51495361],
       [ 129.18315125,  291.10913086,  287.89047241, ...,  613.37158203,
        -149.88583374,  720.54077148],
       [ 218.55038452,  388.26470947,  372.72775269, ...,  677.58569336,
         -51.29973602,  789.12585449]])

In [35]:
data.shape

(4, 1000)

#### Attempting to implement analysis

**Recreating the method described in the study**: [Classification of SSVEP signals using the combined FoCCA-KNN method and comparison with other machine learning methods](https://www.sciencedirect.com/science/article/abs/pii/S1746809423003907)

**Electrode placement**: "OZ", "O1", "O2"


In [ ]:
from brainflow.data_filter import DataFilter, FilterTypes

##### Band-stop and band-pass filters


In [ ]:
sampling_rate = BoardShim.get_sampling_rate(board_id=BoardIds.NEUROPAWN_KNIGHT_BOARD)
DataFilter.perform_bandpass(
    data=data,
    sampling_rate=sampling_rate,
    start_freq=6.0,
    stop_freq=20.0,
    order=2,
    filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
    ripple=0,
)
DataFilter.perform_bandstop(  # is this neccesary? we already have bandpass...
    data=data,
    sampling_rate=sampling_rate,
    start_freq=49.0,
    stop_freq=51.0,
    order=2,
    filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
    ripple=0,
)
DataFilter.perform_bandstop(  # is this neccesary? we already have bandpass...
    data=data,
    sampling_rate=sampling_rate,
    start_freq=59.0,
    stop_freq=61.0,
    order=2,
    filter_type=FilterTypes.BUTTERWORTH_ZERO_PHASE.value,
    ripple=0,
)

##### Common Average Referencing (CAR) Filter


In [ ]:
data -= data.mean()

##### Selecting the appropriate harmonics, channels and coefficients


#### End session


In [36]:
for channel in channels:
    board.config_board(f"choff_{channel}")
    print(f"Disabled channel {channel} with gain {gain_value}.")
    time.sleep(0.5)

Disabled channel 1 with gain 12.
Disabled channel 2 with gain 12.
Disabled channel 3 with gain 12.
Disabled channel 4 with gain 12.


In [37]:
board.stop_stream()
board.release_session()